In [1]:
# 이미지 사이즈 조정
import cv2
import os
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil

In [18]:
# 컬러이미지를 회색조 이미지로 변환하여 다시 저장
def make_gray_images(directory):
    filelist = os.listdir(directory)
    for filename in tqdm(filelist):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # 원하는 확장자로 변경
            file_path = os.path.join(directory, filename)
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            filename = filename.split('.jpg')[0] + '_aug_gray' + '.jpg'
            cv2.imwrite('.....' + filename, img)
                    
            

# 이미지가 저장된 디렉토리 경로
directory_path = '.....'

make_gray_images(directory_path)

100%|██████████| 10659/10659 [06:04<00:00, 29.26it/s]


In [1]:
DATA_PATH = '...' # 원본 이미지 경로
GRAY_PATH = '...' # 회색조 이미지 저장 경로

In [5]:
# 원본 그레이 파일 만들기
for file in tqdm(os.listdir(DATA_PATH + 'labels/')):
    if file.endswith('.txt') and 'aug' not in file:
       aug_file = file.split('.txt')[0] + '_aug_gray.txt'
       shutil.copy('원본 경로', '이동 경로')

100%|██████████| 10658/10658 [00:19<00:00, 533.82it/s]


# 2. 2차 샘플링
- 데이터셋의 크기가 매우 커서 학습 속도 및 업로드 속도가 매우 느린 문제 발생
- 데이터셋의 크기를 줄이는 시도
- 학습 데이터: 약 3,000개
- 검증 데이터: 약 800개

In [24]:
import pandas as pd
df1 = pd.read_csv('/.../sample_train_file.csv') # 1차 학습 샘플링 데이터
df2 = pd.read_csv('/.../sample_val_file.csv') # 1차 검증 샘플링 데이터
display(df1.head())
print(df1.shape, df2.shape)

,code,name,type,pm별 위반 유형,PM,label_name,위반 유형
0,22,S002501_032_0126_C_D_F_0.jpg,위반,자전거 탑승자_보행자도로 통행 위반,자전거,S002501_032_0126_C_D_F_0.json,보행자도로 통행 위반
1,18,S000574_063_0480_C_D_F_0.jpg,위반,오토바이 탑승자_정지선 위반,오토바이,S000574_063_0480_C_D_F_0.json,정지선 위반
2,17,C000890_000_0198_C_D_F_0.jpg,위반,오토바이 탑승자_신호 위반,오토바이,C000890_000_0198_C_D_F_0.json,신호 위반
3,13,S000506_004_0048_C_D_F_0.jpg,정상,오토바이 탑승자,오토바이,S000506_004_0048_C_D_F_0.json,정상
4,23,S000801_033_0036_C_D_F_0.jpg,위반,자전거 탑승자_안전모 미착용 위반,자전거,S000801_033_0036_C_D_F_0.json,안전모 미착용 위반


(10658, 7) (2665, 7)


In [8]:
# 위반 유형 비율에 따라 학습 데이터 재분류
from sklearn.model_selection import train_test_split
tr_s1, tr_s2, y1, y2 = train_test_split(df1.drop(['위반 유형'], axis = 1), df1['위반 유형'], test_size=0.3, stratify=df1['위반 유형'], random_state=42)
tr_sample = pd.concat([tr_s2, y2], axis = 1).reset_index(drop = True)
display(tr_sample.head())
print(tr_sample.shape)


,code,name,type,pm별 위반 유형,PM,label_name,위반 유형
0,19,S001167_009_0080_C_D_F_0.jpg,위반,오토바이 탑승자_횡단보도 주행 위반,오토바이,S001167_009_0080_C_D_F_0.json,횡단보도 주행 위반
1,13,S000476_023_0196_C_N_F_0.jpg,정상,오토바이 탑승자,오토바이,S000476_023_0196_C_N_F_0.json,정상
2,33,F000057_000_7935_C_D_F_1.jpg,위반,킥보드 탑승자_신호 위반,킥보드,F000057_000_7935_C_D_F_1.json,신호 위반
3,20,S001242_087_0064_C_D_F_0.jpg,정상,자전거 탑승자,자전거,S001242_087_0064_C_D_F_0.json,정상
4,13,S001062_000_0432_C_D_F_0.jpg,정상,오토바이 탑승자,오토바이,S001062_000_0432_C_D_F_0.json,정상


(3198, 7)


In [19]:
# 위반 유형 비율에 따라 검증 데이터 재분류
val_s1, val_s2, val_y1, val_y2 = train_test_split(df2.drop(['위반 유형'], axis = 1), df2['위반 유형'], test_size=0.3, stratify=df2['위반 유형'], random_state=42)
val_sample = pd.concat([val_s2, val_y2], axis = 1).reset_index(drop = True)
display(val_sample.head())
print(val_sample.shape)

,code,name,type,pm별 위반 유형,PM,label_name,위반 유형
0,30,T012478_000_0360_C_D_F_1.jpg,위반,킥보드 탑승자_보행자도로 통행 위반,킥보드,T012478_000_0360_C_D_F_1.json,보행자도로 통행 위반
1,36,F000038_000_0200_C_D_F_1.jpg,위반,킥보드 탑승자_동승자 탑승 위반,킥보드,F000038_000_0200_C_D_F_1.json,동승자 탑승 위반
2,30,T015205_026_0240_B_D_F_1.jpg,위반,킥보드 탑승자_보행자도로 통행 위반,킥보드,T015205_026_0240_B_D_F_1.json,보행자도로 통행 위반
3,25,T011772_000_0510_C_D_F_1.jpg,위반,자전거 탑승자_신호 위반,자전거,T011772_000_0510_C_D_F_1.json,신호 위반
4,13,S000523_005_0252_C_N_F_0.jpg,정상,오토바이 탑승자,오토바이,S000523_005_0252_C_N_F_0.json,정상


(800, 7)


In [20]:
# 재분류 후 학습 데이터 컬럼별 비율
display(tr_sample['PM'].value_counts(normalize=True))
display(tr_sample['위반 유형'].value_counts(normalize=True))
display(tr_sample['type'].value_counts(normalize=True))

자전거     0.412445
오토바이    0.387117
킥보드     0.200438
Name: PM, dtype: float64

정상             0.275485
보행자도로 통행 위반    0.173546
안전모 미착용 위반     0.142902
신호 위반          0.119137
횡단보도 주행 위반     0.115385
정지선 위반         0.102251
무단횡단 위반        0.050969
동승자 탑승 위반      0.020325
Name: 위반 유형, dtype: float64

위반    0.724515
정상    0.275485
Name: type, dtype: float64

In [21]:
# 재분류 후 검증 데이터 컬럼별 비율
display(val_sample['PM'].value_counts(normalize=True))
display(val_sample['위반 유형'].value_counts(normalize=True))
display(val_sample['type'].value_counts(normalize=True))

오토바이    0.39625
자전거     0.38875
킥보드     0.21500
Name: PM, dtype: float64

정상             0.27625
보행자도로 통행 위반    0.17375
안전모 미착용 위반     0.14250
신호 위반          0.11875
횡단보도 주행 위반     0.11500
정지선 위반         0.10250
무단횡단 위반        0.05125
동승자 탑승 위반      0.02000
Name: 위반 유형, dtype: float64

위반    0.72375
정상    0.27625
Name: type, dtype: float64

In [22]:
tr_sample.to_csv('...csv', index = False)
val_sample.to_csv('...csv', index = False)

##### 2차 샘플링 된 데이터로 학습 속도를 높이고 에포크 수를 늘려서 깊게 학습해본다.